# Web Scraping

In [8]:
# Importação das bibliotecas
import pandas as pd
import time
import re
import random
import logging
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

In [9]:
# Configuração do navegador
options = webdriver.ChromeOptions()
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-gpu")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)
options.headless = False  # Altere para True se quiser rodar em segundo plano

In [10]:
# Configuração do logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [11]:
def rolar_ate_todos_anuncios(navegador, seletor_card, maximo_tentativas=10, espera_max=5):
    ultimo_total = 0

    for tentativa in range(maximo_tentativas):
        # Rola até o final
        navegador.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        try:
            # Espera até que o número de cards aumente
            WebDriverWait(navegador, espera_max).until(
                lambda driver: len(driver.find_elements(By.CSS_SELECTOR, seletor_card)) > ultimo_total
            )
        except TimeoutException:
            # Nenhum novo card apareceu dentro do tempo de espera
            break

        # Atualiza o total
        ultimo_total = len(navegador.find_elements(By.CSS_SELECTOR, seletor_card))


In [12]:
# Seletores CSS
seletor_card = "li[data-cy='rp-property-cd']"
seletor_preco = "p[class='text-2-25 text-neutral-120 font-semibold']"
seletor_tamanho = "li[data-cy='rp-cardProperty-propertyArea-txt']"
seletor_quartos = "li[data-cy='rp-cardProperty-bedroomQuantity-txt']"
seletor_banheiros = "li[data-cy='rp-cardProperty-bathroomQuantity-txt']"
seletor_estacionamento = "li[data-cy='rp-cardProperty-parkingSpacesQuantity-txt']"
seletor_bairro = 'h2[class="text-ellipsis text-2 font-semibold overflow-hidden font-secondary"]'
seletor_endereco = 'p[data-cy="rp-cardProperty-street-txt"]'


In [13]:
#URL
url_inicial = "https://www.zapimoveis.com.br/venda/apartamentos/mg+belo-horizonte/"

In [14]:
navegador = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(navegador, 10)

# Acessa o site
navegador.get(url_inicial)
time.sleep(3)

# Aceita cookies se necessário
try:
    botao_cookies = wait.until(EC.element_to_be_clickable((By.ID, "adopt-accept-all-button")))
    botao_cookies.click()
    logging.info("Cookies aceitos.")
except (NoSuchElementException, TimeoutException):
    logging.info("Cookies não encontrados ou já aceitos.")

# Extração dos dados
pagina_atual = 1
lista_imoveis = []

while True:
    logging.info(f"Coletando página {pagina_atual}...")

    rolar_ate_todos_anuncios(navegador, seletor_card)

    soup = BeautifulSoup(navegador.page_source, 'html.parser')
    anuncios = soup.select(seletor_card)

    if not anuncios:
        logging.warning("Nenhum imóvel encontrado nesta página.")
        break

    for anuncio in anuncios:
        try:
            # Extrair bairro limpo
            texto_local = anuncio.select_one(seletor_bairro).get_text(strip=True) if anuncio.select_one(seletor_bairro) else None

            bairro = 'N/D'
            if texto_local:
                # Procura "em <bairro>, <cidade>"
                match = re.search(r'em\s*([A-Za-zÀ-ú\s\(\)\-]+),', texto_local)
                if match:
                    bairro = match.group(1).strip()

            dados = {
                "Preço": anuncio.select_one(seletor_preco).get_text(strip=True) if anuncio.select_one(seletor_preco) else 'N/D',
                "Tamanho": anuncio.select_one(seletor_tamanho).get_text(strip=True) if anuncio.select_one(seletor_tamanho) else 'N/D',
                "Quartos": anuncio.select_one(seletor_quartos).get_text(strip=True) if anuncio.select_one(seletor_quartos) else 'N/D',
                "Banheiros": anuncio.select_one(seletor_banheiros).get_text(strip=True) if anuncio.select_one(seletor_banheiros) else 'N/D',
                "Vagas": anuncio.select_one(seletor_estacionamento).get_text(strip=True) if anuncio.select_one(seletor_estacionamento) else 'N/D',
                "Bairro": bairro,
                "Endereço": anuncio.select_one(seletor_endereco).get_text(strip=True) if anuncio.select_one(seletor_endereco) else 'N/D',
            }

            lista_imoveis.append(dados)
        except Exception as e:
            logging.error(f"Erro ao extrair dados de um anúncio: {e}")

    # Avança para próxima página
    try:
        botao_proxima = navegador.find_element(By.CSS_SELECTOR, 'button[data-testid="next-page"]')
        if botao_proxima.is_enabled():
            navegador.execute_script("arguments[0].scrollIntoView(true);", botao_proxima)
            time.sleep(1)
            botao_proxima.click()
            pagina_atual += 1
            time.sleep(random.uniform(2, 6))
        else:
            logging.info("Fim da paginação.")
            break
    except NoSuchElementException:
        logging.info("Botão de próxima página não encontrado.")
        break

# Encerra o navegador
navegador.quit()
logging.info("Navegador encerrado.")

2025-09-26 09:46:06,672 - INFO - ====== WebDriver manager ======


2025-09-26 09:46:07,103 - INFO - Get LATEST chromedriver version for google-chrome
2025-09-26 09:46:07,199 - INFO - Get LATEST chromedriver version for google-chrome
2025-09-26 09:46:07,288 - INFO - There is no [mac64] chromedriver "140.0.7339.207" for browser google-chrome "140.0.7339" in cache
2025-09-26 09:46:07,288 - INFO - Get LATEST chromedriver version for google-chrome
2025-09-26 09:46:07,528 - INFO - WebDriver version 140.0.7339.207 selected
2025-09-26 09:46:07,531 - INFO - Modern chrome version https://storage.googleapis.com/chrome-for-testing-public/140.0.7339.207/mac-arm64/chromedriver-mac-arm64.zip
2025-09-26 09:46:07,531 - INFO - About to download new driver from https://storage.googleapis.com/chrome-for-testing-public/140.0.7339.207/mac-arm64/chromedriver-mac-arm64.zip
2025-09-26 09:46:07,658 - INFO - Driver downloading response is 200
2025-09-26 09:46:08,423 - INFO - Get LATEST chromedriver version for google-chrome
2025-09-26 09:46:08,547 - INFO - Driver has been saved

In [15]:
# Salva em DataFrame e CSV final
base_imoveis = pd.DataFrame(lista_imoveis)
base_imoveis.to_csv("base_bruta_imoveis_zap_belo_horizonte.csv", sep=";", encoding="utf-8-sig", index=False)

In [16]:
logging.info(f"Total de imóveis coletados: {len(base_imoveis)}")
logging.info(f"Páginas navegadas: {pagina_atual}")

2025-09-26 10:48:26,826 - INFO - Total de imóveis coletados: 9990
2025-09-26 10:48:26,827 - INFO - Páginas navegadas: 334


In [17]:
base_imoveis

,Preço,Tamanho,Quartos,Banheiros,Vagas,Bairro,Endereço
0,R$ 1.450.000,Tamanho do imóvel135 m²,Quantidade de quartos4,Quantidade de banheiros2,Quantidade de vagas de garagem2,Gutierrez,Rua Cônego Rocha Franco
1,Valor sob consulta,Tamanho do imóvel62-67 m²,Quantidade de quartos2,Quantidade de banheiros1,N/D,Funcionários,Rua Gonçalves Dias
2,R$ 1.070.000,Tamanho do imóvel115 m²,Quantidade de quartos2,Quantidade de banheiros2,Quantidade de vagas de garagem2,Santo Antônio,Rua Antônio Dias
3,R$ 280.000,Tamanho do imóvel60 m²,Quantidade de quartos3,Quantidade de banheiros1,Quantidade de vagas de garagem1,Estoril,Rua Vereador Nelson Cunha
4,A partir deR$ 1.335.000,Tamanho do imóvel78 m²,Quantidade de quartos3,Quantidade de banheiros2,Quantidade de vagas de garagem2,São Pedro,Rua Viçosa
...,...,...,...,...,...,...,...
9985,R$ 1.350.000,Tamanho do imóvel59 m²,Quantidade de quartos2,Quantidade de banheiros2,Quantidade de vagas de garagem2,Lourdes,Rua Felipe dos Santos
9986,R$ 960.000,Tamanho do imóvel73 m²,Quantidade de quartos2,Quantidade de banheiros2,Quantidade de vagas de garagem2,Barro Preto,Rua dos Timbiras
9987,R$ 550.000,Tamanho do imóvel70 m²,Quantidade de quartos3,Quantidade de banheiros2,Quantidade de vagas de garagem2,Castelo,
9988,R$ 760.000,Tamanho do imóvel80 m²,Quantidade de quartos3,Quantidade de banheiros2,Quantidade de vagas de garagem3,Alto Caiçaras,
